In [1]:
import multiprocessing as mp
import re
import nltk

#### Считываем данные песни и их описание(весит больше 100 мб)

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('C:/Users/Win 10/Desktop/lyrics-data.csv')
data.head()

,ALink,SName,SLink,Lyric,Idiom
0,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH
1,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH
2,/10000-maniacs/,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH
3,/10000-maniacs/,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH
4,/10000-maniacs/,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,ENGLISH


In [4]:
data = data[['SName','Lyric']]

#### Предобработка

In [5]:
data.isnull().sum()

SName     0
Lyric    38
dtype: int64

In [6]:
data = data.dropna()
data.isnull().sum()

SName    0
Lyric    0
dtype: int64

In [7]:
data.shape

(209484, 2)

In [8]:
data = data.drop_duplicates()

In [9]:
data.shape

(166660, 2)

In [10]:
data = data.reset_index(drop=True)

In [11]:
data['text'] = data['SName'] + " " + data['Lyric']

In [12]:
text = str(data['text'])

In [13]:
def clean(text):
    text = text.lower()
    text = re.sub("[0-9]", "", text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

text = clean(text)    
sentences = nltk.sent_tokenize(text)
words = [nltk.word_tokenize(word) for word in sentences]

In [14]:
def cols_prep(df):
    pool = mp.Pool(4)
    df['prep'] = pool.map(clean, df['all'])
    pool.close()

In [15]:
%%timeit -n 1 -r 1
cols_prep

600 ns ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [16]:
from gensim.models import Word2Vec

In [17]:
word2vec = Word2Vec(words, min_count=2)

In [18]:
word2vec.wv.most_similar(1,topn=10)

[('i', 0.13720960915088654),
 ('these', 0.06835115700960159),
 ('v', 0.03366147726774216),
 ('cad', 0.00947623886168003),
 ('days', 0.008378744125366211),
 ('zeca', 0.004474645014852285),
 ('the', -0.0035628369078040123),
 ('vou', -0.010686768218874931),
 ('are', -0.023580506443977356),
 ('h', -0.09579785168170929)]